# MATH50003 Numerical Analysis (2022–23)
# Lab 5: Orthogonal Matrices

This lab explores orthogonal matrices, including permutations and reflections.
We will construct special types to capture the structure of these orthogonal operations,
With the goal of implementing fast matrix*vector and matrix\vector operations.

In [1]:
using LinearAlgebra, Test
import Base: getindex, *, size, \

------

**Problem 1** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` in $O(1)$ operations and `*` in $O(n)$ operations,
where $n$ is the length of the permutation.

In [2]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    # This is an internal constructor: allows us to check validity of the input.
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

function size(P::PermutationMatrix)
    (length(P.p),length(P.p))
end

# getindex(P, k, j) is a synonym for P[k,j]
function getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Implement P[k,j]
    (P.p[k] == j) ? 1 : 0
    

end
function *(P::PermutationMatrix, x::AbstractVector)
    # TODO: return a vector whose entries are permuted according to P.p
    x[P.p]
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]

n = 100_000
p = Vector(n:-1:1) # makes a Vector corresponding to [n,n-1,…,1]
P = PermutationMatrix(p)
x = randn(n)
@test P*x == x[p]

Test Passed

-------

**Problem 2.1** Complete the implementation of a type representing an n × n
reflection that supports `Q[k,j]` in $O(1)$ operations and `*` in $O(n)$ operations.
The reflection may be complex (that is, $Q ∈ U(n)$ is unitary).

In [3]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

function size(Q::Reflection)
    (length(Q.v),length(Q.v))
end

# getindex(Q, k, j) is synonym for Q[k,j]
function getindex(Q::Reflection, k::Int, j::Int)
    # TODO: implement Q[k,j] == (I - 2v*v')[k,j] but using O(1) operations.
    # Hint: the function `conj` gives the complex-conjugate
    if k == j
        return 1 - 2Q.v[k] * conj(Q.v[j])
    else
        return -2Q.v[k] * conj(Q.v[j])
    end
    

end
function *(Q::Reflection, x::AbstractVector)
    # TODO: implement Q* x, equivalent to (I - 2v*v')*x but using only O(n) operations
    #dot literally has a function
    #this formula is in the notes since finding Q would be nn so just doing this is n
    #ie Qx = x - 2v(v.x)
    x - 2 * Q.v * dot(Q.v, x)

end

# If your code is correct, these "unit tests" will succeed
n = 10
x = randn(n) + im*randn(n)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q'Q ≈ I
n = 100_000
x = randn(n) + im*randn(n)
Q = Reflection(x)
v = x/norm(x)
@test Q*v ≈ -v

Test Passed

**Problem 2.2** Complete the following implementation of a Housholder reflection  so that the
unit tests pass, using the `Reflection` type created above.
Here `s == true` means the Householder reflection is sent to the positive axis and `s == false` is the negative axis.

In [4]:
function householderreflection(s::Bool, x::AbstractVector)
    #im assuming this is real
    #the thing of just e1 normx sign is if then apply the reflection to another vector
    #here add since just finding the reflection
    # TODO: return a `Reflection` corresponding to a Householder reflection
    #REFLECT ONTO POS X MEANS SIGN OF NORM(X) IS NEG!!!
    sign = s ? -1 : 1
    #use copy to keep original value
    y = copy(x)

    y[1] += (sign * norm(x))
    #putting into reflection constructor
    Reflection(y)

end

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ [norm(x);zeros(eltype(x),length(x)-1)]

Q = householderreflection(false, x)
@test Q isa Reflection
@test Q*x ≈ [-norm(x);zeros(eltype(x),length(x)-1)]

Test Passed

---------

**Problem 3**
Complete the definition of `Reflections` which supports a sequence of reflections,
that is,
$$
Q = Q_{𝐯_1} ⋯ Q_{𝐯_m}
$$
where the vectors are stored as a matrix $V ∈ ℂ^{n × m}$ whose $j$-th column is $𝐯_j∈ ℂ^n$, and
$$
Q_{𝐯_j} = I - 2 𝐯_j 𝐯_j^⋆
$$
is a reflection.

In [18]:
struct Reflections{T} <: AbstractMatrix{T}
    V::Matrix{T}
end

size(Q::Reflections) = (size(Q.V,1), size(Q.V,1))


function *(Q::Reflections, x::AbstractVector)
    # TODO: Apply Q in O(mn) operations by applying
    # the reflection corresponding to each column of Q.V to x
    #does order matter???
    m, n = size(Q.V)
    #formula for vector is x - 2v(v.x)   v is unit
    #carry = I(n)
    #start from n since apply nth column to x first and work backwards
    for i = n:-1:1
        #@show i
        #order matters so can't do *= because need x to be later
        #CAREFUL WITH MULTIPLICATION ORDER WITH VECTORS AND MATRIX ETC
        x = Reflection(Q.V[:,i]) * x
        
    end

    x
end

function getindex(Q::Reflections, k::Int, j::Int)
    # TODO: Return Q[k,j] in O(mn) operations (hint: use *)
    # Reflections is a matrix
    # SOLUTION
    T = eltype(Q.V)
    #size of the reflection is just the size of the first column 
    #(this will be the dimension of the resulting matrix thats all their combination)
    m,n = size(Q)
    ej = zeros(T, m) 
    ej[j] = one(T)
    
    #ie get kth row of jth column of of the resulting matrix
    #why is this any different to Q[k, j] => this is what we are currently defining so would just be
    #recursive to do this
    #have already defined it for a reflection vector so just get the jth column vector and get kth element
    return (Q*ej)[k]

    
    #want index for the resulting total of the reflections in each column of V
    #dont overrcomplicate getting index literally turn into something can already get and get particular part 
end

Y = randn(5,3)
V = Y * Diagonal([1/norm(Y[:,j]) for j=1:3])
#so i guess this ccompares equality by checking size the same and each element equal 
#so if define get index wierdly like us then this below makes sense for checking the matrix Q is 
#equivalent to the reflections of each column of V
Q = Reflections(V)
@test Q ≈ (I - 2V[:,1]*V[:,1]')*(I - 2V[:,2]*V[:,2]')*(I - 2V[:,3]*V[:,3]')
@test Q'Q ≈ I

Error During Test at In[18]:54
  Test threw exception
  Expression: Q ≈ (I - (2 * V[:, 1]) * (V[:, 1])') * (I - (2 * V[:, 2]) * (V[:, 2])') * (I - (2 * V[:, 3]) * (V[:, 3])')
  StackOverflowError:
  Stacktrace:
    [1] Array
      @ .\boot.jl:459 [inlined]
    [2] Array
      @ .\boot.jl:468 [inlined]
    [3] zeros
      @ .\array.jl:588 [inlined]
    [4] zeros
      @ .\array.jl:584 [inlined]
    [5] getindex(Q::Reflections{Float64}, k::Int64, j::Int64)
      @ Main .\In[18]:35
    [6] getindex(Q::Reflections{Float64}, k::Int64, j::Int64) (repeats 52167 times)
      @ Main .\In[18]:41
    [7] _getindex
      @ .\abstractarray.jl:1291 [inlined]
    [8] getindex
      @ .\abstractarray.jl:1241 [inlined]
    [9] _broadcast_getindex
      @ .\broadcast.jl:636 [inlined]
   [10] _getindex
      @ .\broadcast.jl:666 [inlined]
   [11] _broadcast_getindex
      @ .\broadcast.jl:642 [inlined]
   [12] getindex
      @ .\broadcast.jl:597 [inlined]
   [13] macro expansion
      @ .\broadcast.jl:96

LoadError: [91mThere was an error during testing[39m

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*